In [31]:
%serialconnect to --port=/dev/ttyUSB1 --baud=115200

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [32]:
# All imports 
import usocket as socket
from machine import Pin
import network
import esp
import utime
import ujson
import gc
import socket
import uhashlib
import ubinascii

esp.osdebug(None)
gc.collect()

In [33]:
sta_if = network.WLAN(network.STA_IF)
ap_if = network.WLAN(network.AP_IF)
print('Station Up: ', sta_if.active())
print('Acess Point Up: ', ap_if.active())

Station Up:  False
Acess Point Up:  False


In [34]:

def connectX(typeX):
    typeX.active(True)
    print('Station Up: ', sta_if.active())
    print('Acess Point Up: ', ap_if.active())


def scanX():
    print('Status: %s\n' % sta_if.status())
    station_default = False
    stations_list = sta_if.scan()
    print('\ntest1: ', stations_list)
    for i, n in enumerate(stations_list, start=0):
        if n[0] == b'SM-A1kerr' or n[0] == b'Atalyah':
            print('\nFOUND SID!: ', n[0])
            station_default = True

    if station_default is False:
        print('\n*** PLEASE ENABLE A DEFAULT STATION! ***')
    return stations_list


# Make a dictionary out of a byte Object b''
def get_meta(c):
    c = c.decode("utf-8")
    holdX = c.split('\r\n')
    print(type(holdX))
    holdy = {}
    for l in holdX:
        dataX = len(l.split(':'))
        if dataX > 1:
            holdy[str(l.split(':')[0])] = str(l.split(':')[1])
        else:
            holdy[str(l.split(':')[0])] = str(l.split(':')[0])

    return holdy


In [35]:
connectX(sta_if)
stations_list = []


if sta_if.active():
    stations_list = scanX()
elif ap_if.active():
    print('Acess Point Up: ', ap_if.active())

Station Up:  True
Acess Point Up:  False
Status: 1000


test1:  [(b'VIRGIN463', b'\xa8\x9a\x93\x1d\xf8\x9e', 1, -41, 3, False), (b'Atalyah', b'\x08`n\xbc\x02\t', 6, -41, 3, False), (b'ASUS2_4_PRIMETIME_905', b'\x08`n\xbc\x02\x08', 6, -42, 3, False), (b'dlink_2_4GHz_PrimeTime', b'@\x9b\xcd\xa0ih', 11, -49, 4, False), (b'BELL366', b'\x90r\x82\xfe\x8e&', 6, -67, 3, False), (b'BELL915', b'@\xf2\x01\xf8\x94W', 1, -75, 3, False), (b'C3D998B14F48', b'\\\x8f\xe0\x04\xe1\xcb', 1, -76, 3, False), (b'Family1', b'B\xf2\x01\xf8\x94W', 1, -76, 3, False), (b'dlink-B2F6', b'\xf8\xe9\x03\xb8\xb2\xf6', 1, -80, 4, False), (b'SmartRG-bb48', b'\xe8,m\x10\xbbL', 11, -93, 3, False)]

FOUND SID!:  b'Atalyah'


In [36]:
print('Connecting to: {0}'.format(stations_list[0][0]))
while not (sta_if.isconnected()):
    print('Trying to connet to server!')
#     if 'Atalyah' in stations_list : 
#     print(stations_list[0][0])
    for x in stations_list:
#         print(x)
        if b'Atalyah' in x or b'SM-A1kerr' in x:
            print(x[0])
            sta_if.connect(x[0], '9054830760')
    utime.sleep(1)


print('Status: {0}\n'.format(sta_if.status(), sta_if.isconnected()),
      ('\nIs connected: {0}'.format(sta_if.isconnected())),
      ('\nConnected to: {0}'.format(sta_if.config('essid'))),
      ('\nifconfig: {0}'.format(sta_if.ifconfig()))
      )

Connecting to: b'VIRGIN463'
Trying to connet to server!
b'Atalyah'
Trying to connet to server!
b'Atalyah'
Status: 1010
 
Is connected: True 
Connected to: Atalyah 
ifconfig: ('192.168.1.240', '255.255.255.0', '192.168.1.1', '192.168.1.1')


<br>
<h1>Websocket Return Header Format</h1>
<p>
HTTP/1.1 101 Switching Protocols<br>
Upgrade: websocket<br>
Connection: Upgrade<br>
Sec-WebSocket-Accept: s3pPLMBiTxaQ9kYGzzhZRbK+xOo=<br>
(remember each header line ends with \r\n and put an extra \r\n after the last one to indicate the end of the header)
</p>
<p>
-hashed key- The server takes the value of Sec-WebSocket-Key sent in handshake request and then appends Globally Unique Identifier (GUID, [RFC4122]) “258EAFA5-E914-47DA-95CA-C5AB0DC85B11” takes SHA-1 of the new value, and is then base64 encoded. This is unlikely to be used by network endpoints that do not understand the WebSocket Protocol.
</p>

In [7]:
print(socket.getaddrinfo('', 80))

[(2, 1, 0, '0.0.0.0', ('0.0.0.0', 80))]


In [37]:
# Complete project details at https://RandomNerdTutorials.com


addr_socket = socket.getaddrinfo('', 80)[0][4]
# addr_socket = socket.getaddrinfo('192.168.43.34', 80)[0][4]
# addr = socket.getaddrinfo('192.168.43.34', 80)
led = Pin(2, Pin.OUT)
key_tag = 'Sec-WebSocket-Key'
guid = b'258EAFA5-E914-47DA-95CA-C5AB0DC85B11'


def web_page():
    if led.value() == 1:
        gpio_state = "ON"
    else:
        gpio_state = "OFF"

    html = """<html><head> <title>ESP Web Server</title> <meta name = "viewport"
    content="width=device-width, initial-scale=1">
    <link rel="icon" href="data:,"> <style>html{font-family: Helvetica;
    display:inline-block; margin: 0px auto; text-align: center;}
    h1{color: #0F3376; padding: 2vh;}p{font-size: 1.5rem;}.button{display:
    inline-block; background-color: #e7bd3b; border: none;
    border-radius: 4px; color: white; padding: 16px 40px; text-decoration:
    none; font-size: 30px; margin: 2px; cursor: pointer;}
    .button2{background-color: #4286f4;}</style></head><body>
    <h1>ESP Web Server</h1>
    <p>GPIO state: <strong>""" + gpio_state + """</strong></p><p>
    <a href="/?led=on"><button class="button">ON</button></a></p>
    <p><a href="/?led=off"><button class="button button2">OFF</button>
    </a></p></body></html>"""
    return html


html2 = """<!DOCTYPE html>
<html>
<head> <title>ESP8266 Pins</title> </head>
<body> <h1>ESP8266 Pins</h1>
    <table border="1"> <tr><th>Pin</th><th>Value</th></tr> %s </table>
</body>
</html>
"""

s=socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind(('', 80))
print('Bind: ', addr_socket)
# print('Bind: %s' % addr[0])
# print('Bind: %s' % addr[0][-1])
s.listen(5)

while True:
    conn, addr = s.accept()
    print('Got a connection!: ')
    print('Got a connection from %s' % str(addr))
    request=conn.recv(1024)
#     conn.send(html2)
#     request = request.decode()
    request_meta = get_meta(request)
    print('Content = %s' % request_meta)
    if key_tag in request_meta:
        print('Sec-Weboscket-Key: ', request_meta[key_tag])
        key_hash = uhashlib.sha1(guid)
        key_hash.update(request_meta[key_tag])
#         key_hash = uhashlib.sha1(request_meta[key_tag])
#         key_hash.update(guid)
        key_sha1 = key_hash.digest()
        key_sha1 = ubinascii.b2a_base64(key_sha1)
        header1 = 'HTTP/1.1 101 Switching Protocols\r\n'
        header2 = 'Upgrade: websocket\r\n'
        header3 = 'Connection: Upgrade\r\n'
        header_sec = 'Sec-WebSocket-Accept: {0}'.format(key_sha1)
        header_end = '\r\n\r\n'
        socketX_upgrade = bytes(header1+header2+header3+header_sec+header_end, 'utf8')
        print('SHA1 Key: ', key_sha1)
        conn.send(socketX_upgrade)
    else:
        print('No Key Found')
          
    
    conn.close()

print('Server closed: ')

.Got a connection!: 
Got a connection from ('192.168.1.208', 45362)
<class 'list'>
Content = {'User-Agent': ' Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.106 Safari/537.36', 'Accept': ' text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9', 'Connection': ' keep-alive', 'Cache-Control': ' max-age=0', 'Accept-Language': ' en-GB,en-US;q=0.9,en;q=0.8,da;q=0.7', 'Accept-Encoding': ' gzip, deflate', 'Host': ' 192.168.1.240', 'GET / HTTP/1.1': 'GET / HTTP/1.1', 'Upgrade-Insecure-Requests': ' 1', '': ''}
No Key Found
..............

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 52, in <module>
KeyboardInterrupt: 


<br>
<p>
Format of getaddrinfo:  [(2, 1, 0, '192.168.43.34', ('192.168.43.34', 80))]
</p>